## Scorecard reader

semi-uniform csv files are read into pandas dataframes, manipulated and added to an sql server database.

this is all datapreprocessing with the following analytical and visualization goals in mind.



In [1]:
#import libraries

import pandas as pd
import numpy as np
import os


In [2]:
def read_csv() -> pd.DataFrame:
    file = '../data/scorecards/UDisc Scorecards.csv'
    rawdata = pd.read_csv(file)
    return rawdata

In [3]:
rawdata = read_csv()
rawdata

,PlayerName,CourseName,LayoutName,Datum,Gesamt,+/-,Hole1,Hole2,Hole3,Hole4,...,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18,Hole19,Hole20,Hole21,Hole22
0,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Julian,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,44,12.0,4,4,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,28,-4.0,2,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,27,-5.0,2,4,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Sina,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,70,16.0,4,5,3,3,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
322,Basti,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,74,20.0,4,8,4,4,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
323,Jörg,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,63,9.0,3,4,4,4,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
324,Kevin,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,50,-4.0,2,4,3,2,...,2.0,3.0,2.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN


In [4]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PlayerName  326 non-null    object 
 1   CourseName  326 non-null    object 
 2   LayoutName  326 non-null    object 
 3   Datum       326 non-null    object 
 4   Gesamt      326 non-null    int64  
 5   +/-         245 non-null    float64
 6   Hole1       326 non-null    int64  
 7   Hole2       326 non-null    int64  
 8   Hole3       326 non-null    int64  
 9   Hole4       326 non-null    int64  
 10  Hole5       326 non-null    int64  
 11  Hole6       326 non-null    int64  
 12  Hole7       326 non-null    int64  
 13  Hole8       326 non-null    int64  
 14  Hole9       326 non-null    int64  
 15  Hole10      301 non-null    float64
 16  Hole11      232 non-null    float64
 17  Hole12      232 non-null    float64
 18  Hole13      193 non-null    float64
 19  Hole14      193 non-null    f

Each csv file has between 7 and 33 columns. the first seven columns are fixed 


0   "PlayerName"      object    
<--- Every round consists of a Par row, followed by
                                               Team rows. Each row after a Par row, represents
                                               a Team of size 1 or higher.
                                               Each Team is represented by a Characters of a String seperated by a '+' ("Player1" or "Player1 + Player2" etc.)
                                             

1   "CourseName"      object

2   "LayoutName"      object

3   "Datum"           object    <--- localized translation of columns name! use iloc|

4   "Gesamt"          int64     <--- localized translation of columns name! use iloc|
this column contains the "PAR" value for the Layout/Course if PlayerName is PAR or a number representing the score of the Team.

5   "+/-"             float64   <--- contains NaN when column Playername contains 'Par'
contains the difference between "GESAMT" of the Team and the "GESAMT" of the corresponding "PAR"

6   "Hole1"           int64     <--- the First Hole of the Course (minimum)


Columns 8-33 are Hole2 - Hole27

the CSV file represents an archive of scorecards.

Played on different Layouts/Courses, with different number of holes and different PAR values per Hole.

The number of columns of the dataframe depends on the longest course (with the highest number of holes) that is saved in the csv file.

Each round that is played (a par row, follewed by 1-10 Team rows) 


The Hole columns contains values from 0 upwards (very very rarely higher than 15) that have two meanings. If "PlayerName" is "Par", then the number represents the Par value for that Hole on that Layout. If "PlayerName" contains a Teamnamestring, then the number represents the score, the player had on that hole during that unique round.

0 is the default value if the scorecard has been created, but no score was entered. 
a scorecard containing a 0 is therfore an unfinished scorecard

1 equals an ACE and is the best possible score.
all other scores (2 and upwards) are named in their relation to the corresponding Par value.

In [5]:
ScoreNaming= pd.DataFrame([['Ace','Ace','Ace','Ace'],
                  ['Birdie','Eagle','Albatross', None],
                  ['Par','Birdie','Eagle','Albatross'],
                  ['Bogey','Par','Birdie','Eagle'],
                  ['Double Bogey','Bogey','Par','Birdie'],
                  ['Tripple Bogey','Double Bogey','Bogey','Par'],
                  ['Quadruple Bogey','Tripple Bogey','Double Bogey5','Bogey'],
                  ['etc.', 'etc.', 'etc.', 'etc.']],
                 index=[1, 2, 3, 4, 5, 6, 7, 8],
                 columns=['Par 3','Par 4','Par 5','Par 6'])
ScoreNaming

,Par 3,Par 4,Par 5,Par 6
1,Ace,Ace,Ace,Ace
2,Birdie,Eagle,Albatross,None
3,Par,Birdie,Eagle,Albatross
4,Bogey,Par,Birdie,Eagle
5,Double Bogey,Bogey,Par,Birdie
6,Tripple Bogey,Double Bogey,Bogey,Par
7,Quadruple Bogey,Tripple Bogey,Double Bogey5,Bogey
8,etc.,etc.,etc.,etc.


All custom scorecards are ignored. They can be identified through the 'LayoutName'

In [6]:
def clear_custom(rawdata: pd.DataFrame) -> pd.DataFrame:
    return rawdata.loc[rawdata.loc[:,'LayoutName']!='Custom',:]

rawdata = clear_custom(rawdata)
rawdata

,PlayerName,CourseName,LayoutName,Datum,Gesamt,+/-,Hole1,Hole2,Hole3,Hole4,...,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18,Hole19,Hole20,Hole21,Hole22
0,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Julian,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,44,12.0,4,4,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,28,-4.0,2,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,27,-5.0,2,4,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Sina,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,70,16.0,4,5,3,3,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
322,Basti,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,74,20.0,4,8,4,4,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
323,Jörg,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,63,9.0,3,4,4,4,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
324,Kevin,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,50,-4.0,2,4,3,2,...,2.0,3.0,2.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN


all Scorecards with 0 values on at least one hole (unfinished score cards) are ignored


In [7]:
#list(rawdata.columns[6:]) <-- all following hole columns from hole 1 to the maximum in the csv file
#rawdata.loc[rawdata.loc[:,rawdata.columns[6]]==2,:]  #all scorecards with a score of 2 on hole1

def clear_unfinished(rawdata: pd.DataFrame) -> pd.DataFrame:
    for i in range(6, rawdata.columns.size):
        #print(f"Zero Values on Hole {i-5}")
        #print(rawdata.loc[rawdata.loc[:,rawdata.columns[i]]==0,:])
        #print('*'*50)
        rawdata = rawdata.loc[rawdata.loc[:,rawdata.columns[i]]!=0,:]
    return rawdata

rawdata = clear_unfinished(rawdata)
rawdata


,PlayerName,CourseName,LayoutName,Datum,Gesamt,+/-,Hole1,Hole2,Hole3,Hole4,...,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18,Hole19,Hole20,Hole21,Hole22
0,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Julian,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,44,12.0,4,4,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,28,-4.0,2,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,27,-5.0,2,4,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Sina,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,70,16.0,4,5,3,3,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
322,Basti,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,74,20.0,4,8,4,4,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
323,Jörg,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,63,9.0,3,4,4,4,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
324,Kevin,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,50,-4.0,2,4,3,2,...,2.0,3.0,2.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN


In [8]:
def clear_lowpar(rawdata: pd.DataFrame) -> pd.DataFrame:
    for i in range(6, rawdata.columns.size):
        matchme = rawdata.loc[(rawdata.iloc[:,0]=='Par')
                          &(rawdata.loc[:,rawdata.columns[i]]<3),:].iloc[:,[1,2,3]]
        if len(matchme.index) > 0:
            filter_ = rawdata.loc[((rawdata.iloc[:,1]==matchme.values[0][0])
                                    &(rawdata.iloc[:,2]==matchme.values[0][1])
                                    &(rawdata.iloc[:,3]==matchme.values[0][2])),:]

            rawdata.drop(index=filter_.index, inplace=True)
        else:
            continue
    return rawdata

rawdata = clear_lowpar(rawdata)
rawdata

,PlayerName,CourseName,LayoutName,Datum,Gesamt,+/-,Hole1,Hole2,Hole3,Hole4,...,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18,Hole19,Hole20,Hole21,Hole22
0,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Julian,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,44,12.0,4,4,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,28,-4.0,2,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Par,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,32,NaN,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,27,-5.0,2,4,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Sina,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,70,16.0,4,5,3,3,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
322,Basti,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,74,20.0,4,8,4,4,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
323,Jörg,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,63,9.0,3,4,4,4,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
324,Kevin,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,50,-4.0,2,4,3,2,...,2.0,3.0,2.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN


now it is time to prepare the rawdata to be split into pardata and scoredata

since the "Par" rows are often redundant, duplicates are dropped and only the 

While each Course/Layout combination is unique, a Layout can change it's Par over the course of time, 
because the circumstances of a Hole change and the Par is adjusted as a result.



In [9]:
rawdata.loc[(rawdata.loc[:,'CourseName']=='Fröndenberg')
            &(rawdata.loc[:,'PlayerName']=='Par')
            ,['Datum','Gesamt','+/-','Hole1','Hole2','Hole3','Hole4','Hole5','Hole6']]

,Datum,Gesamt,+/-,Hole1,Hole2,Hole3,Hole4,Hole5,Hole6
227,2019-08-17 12:40,28,NaN,3,3,3,3,4,3
231,2019-08-17 11:39,28,NaN,3,3,3,3,4,3
238,2019-06-22 19:21,27,NaN,3,3,3,3,3,3


The above example shows, that sometime between the 22nd June 2019 and 17th August 2019 the Par Value for Hole5 was adjusted
from a Par 3 to a Par 4. The total Par value for the Layout increased from 27 to 28.


In [10]:
def create_pardata (rawdata: pd.DataFrame) -> pd.DataFrame:
    pardata = rawdata[rawdata['PlayerName'] == 'Par']

    old_new = pardata.groupby(by=list(pardata.columns[[1,2,4]])).agg(oldest=(pardata.columns[3],'min'),
                                                       newest=(pardata.columns[3],'max'))

    pardata.drop_duplicates(subset=[pardata.columns[1],
                                    pardata.columns[2],
                                    pardata.columns[4]], 
                            inplace=True, ignore_index=False)
    pardata = pardata.sort_values(by=list(pardata.columns[[1,2,4]])).drop(columns=pardata.columns[[0,3,5]])
    pardata.reset_index(inplace=True, drop=True)
    pardata.insert(loc=2, column='Oldest', value=pd.Series(old_new['oldest'].values))
    pardata.insert(loc=3, column='Newest', value=pd.Series(old_new['newest'].values))
    return pardata
pardata = create_pardata(rawdata)

<ipython-input-10-6ada0fe9a1ce>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pardata.drop_duplicates(subset=[pardata.columns[1],


In [11]:
pardata

,CourseName,LayoutName,Oldest,Newest,Gesamt,Hole1,Hole2,Hole3,Hole4,Hole5,...,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18,Hole19,Hole20,Hole21,Hole22
0,Beckum Phoenix Park,Main,2017-03-24 12:34,2017-03-24 12:34,36,3,3,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bochum Volkspark Langendreer,NORMAL,2019-08-03 12:58,2021-03-24 16:14,32,3,4,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dortmund Revierpark Wischlingen,Main - Altes Layout,2017-05-14 10:10,2017-05-14 10:10,54,3,3,3,3,3,...,3.0,3.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
3,Dortmund Revierpark Wischlingen,Standard Layout,2017-01-28 11:46,2017-01-28 11:46,54,3,3,3,3,3,...,3.0,3.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
4,Dortmund Revierpark Wischlingen,Standard Layout,2017-07-17 17:05,2020-07-26 11:46,55,3,4,3,3,3,...,3.0,3.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
5,Fröndenberg,Main,2019-06-22 19:21,2019-06-22 19:21,27,3,3,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fröndenberg,Main,2019-08-17 11:39,2019-08-17 12:40,28,3,3,3,3,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Herne Gysenbergpark,Main,2017-07-24 17:53,2018-12-23 11:23,27,3,3,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Lünen Alte Ziegelei,Main,2020-11-08 14:33,2020-11-08 14:33,37,3,3,4,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Lünen Seepark,Main,2020-07-28 14:52,2020-07-28 14:52,37,3,4,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def add_HoleCount(pardata: pd.DataFrame) -> pd.DataFrame:
    pardata.insert(loc=4, column='HoleCount', value=pardata.loc[:,pardata.columns[5:]].count(axis='columns'))
    return pardata

pardata = add_HoleCount(pardata)

#pardata.iloc[:,:]
#df.count(axis='columns')

In [13]:
def clear_spaces(pardata: pd.DataFrame) -> pd.DataFrame:
    pardata.iloc[:,0] = pardata.iloc[:,0].apply(lambda x: x.strip())
    pardata.iloc[:,1] = pardata.iloc[:,1].apply(lambda x: x.strip())
    return pardata

In [14]:
pardata = clear_spaces(pardata)

The Pardata is now ready to be written onto the SQL Database. The writing proccess is handled in another script.


In [15]:
def create_scoredata (rawdata: pd.DataFrame) -> pd.DataFrame:
    scoredata = rawdata[rawdata['PlayerName'] != 'Par']
    return scoredata
scoredata = create_scoredata(rawdata)
scoredata

,PlayerName,CourseName,LayoutName,Datum,Gesamt,+/-,Hole1,Hole2,Hole3,Hole4,...,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18,Hole19,Hole20,Hole21,Hole22
1,Julian,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,44,12.0,4,4,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,28,-4.0,2,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,27,-5.0,2,4,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 09:55,26,-6.0,2,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-02-28 09:27,27,-5.0,3,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Sina,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,70,16.0,4,5,3,3,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
322,Basti,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,74,20.0,4,8,4,4,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
323,Jörg,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,63,9.0,3,4,4,4,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
324,Kevin,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,50,-4.0,2,4,3,2,...,2.0,3.0,2.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN


In [16]:
def add_Teamsize(scoredata: pd.DataFrame) -> pd.DataFrame:
    scoredata.insert(loc=4, column='TeamSize', value=scoredata['PlayerName'].apply(lambda x: len(x.split('+'))))
    scoredata.drop(columns='+/-', inplace=True)
    return scoredata
scoredata = add_Teamsize(scoredata)
scoredata

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,PlayerName,CourseName,LayoutName,Datum,TeamSize,Gesamt,Hole1,Hole2,Hole3,Hole4,...,Hole13,Hole14,Hole15,Hole16,Hole17,Hole18,Hole19,Hole20,Hole21,Hole22
1,Julian,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,1,44,4,4,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-24 16:14,1,28,2,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 10:33,1,27,2,4,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-03-07 09:55,1,26,2,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Dennis B.,Bochum Volkspark Langendreer,NORMAL,2021-02-28 09:27,1,27,3,4,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Sina,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,1,70,4,5,3,3,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
322,Basti,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,1,74,4,8,4,4,...,3.0,4.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN
323,Jörg,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,1,63,3,4,4,4,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
324,Kevin,Rennbahnpark Neuss,New 18 Hole,2016-05-26 15:57,1,50,2,4,3,2,...,2.0,3.0,2.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
